# European Centre for Medium Range Weather Forecasts

Presentado por: Yosef Guevara Salamanca

## 1. Descarga del script

[Download script](https://gist.github.com/mrocklin/26d8323f9a8a6a75fce0)

Download historical global weather data from the ECMWF.

In [ ]:
import pandas as pd
dates = pd.date_range('2014-01-01', '2014-12-31', freq='D')
dates = [str(d).split()[0] for d in dates]

from ecmwfapi import ECMWFDataServer

server = ECMWFDataServer()

for date in dates:
    server.retrieve({
      'stream'    : "oper",
      'levtype'   : "sfc",
      'param'     : "165.128/166.128/167.128",
      'dataset'   : "interim",
      'step'      : "00",
      'grid'      : "0.25/0.25",
      'time'      : "00/06/12/18",
      'date'      : date,
      'type'      : "an",
      'class'     : "ei",
      'target'    : date + ".nc3",
      'format'    : "netcdf" })

**Try the following:**

*  What is the variance in temperature over time?
*  What areas experienced the largest temperature swings in the last month relative to their previous history?
*  Plot the temperature of the earth as a function of latitude and then as longitude

In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import dask.array as da
from datetime import date, timedelta
import datetime as dt

In [2]:
from dask.distributed import Client
client = Client(n_workers=2)

In [3]:
files = sorted(glob('2014-*.nc3'))
dss=[nc.Dataset(fns)['t2m'] for fns in files[:90]]
arrays = [da.from_array(dst, chunks=(4,400,500)) for dst in dss]

x=da.concatenate(arrays, axis=0)
x=x.rechunk('auto')
x

ValueError: Need array(s) to concatenate